**# 自定义层**

* 可以用创造性的方式组合不同的层，从而设计出适用于各种任务的架构。

**1. 构造一个没有任何参数的自定义层**

In [2]:
import torch
import torch.nn.functional as F
from torch import nn


class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [3]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

* 可以将层作为组件合并到更复杂的模型中。

In [4]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [5]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-3.7253e-09, grad_fn=<MeanBackward0>)

**2. 带参数的层**

* 如下实现自定义版本的全连接层。 该层需要两个参数，一个用于表示权重，另一个用于表示偏置项。 在此实现中，使用修正线性单元作为激活函数。该层需要输入参数：in_units和units，分别表示输入数和输出数。

In [11]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [12]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-0.4347,  1.3235,  0.5923],
        [-0.7439, -1.0569, -0.5721],
        [ 0.2388,  0.5794,  0.3254],
        [ 0.9396,  0.8565, -1.4810],
        [ 0.5687,  0.8050,  0.1317]], requires_grad=True)

In [13]:
linear(torch.rand(2, 5))

tensor([[0.7983, 1.6391, 0.0000],
        [1.3348, 1.9794, 0.0000]])

In [14]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.],
        [0.]])